In [1]:
import pandas as pd
import numpy as np

In [2]:
accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')[['loan_amnt','issue_d','title','last_fico_range_high',
                                                       'last_fico_range_low','annual_inc','installment',
                                                       'zip_code','addr_state','emp_length','policy_code']]
rejected = pd.read_csv('rejected_2007_to_2018Q4.csv')

C:\Users\aznan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
accepted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   loan_amnt             float64
 1   issue_d               object 
 2   title                 object 
 3   last_fico_range_high  float64
 4   last_fico_range_low   float64
 5   annual_inc            float64
 6   installment           float64
 7   zip_code              object 
 8   addr_state            object 
 9   emp_length            object 
 10  policy_code           float64
dtypes: float64(6), object(5)
memory usage: 189.7+ MB


In [4]:
rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Amount Requested      1048575 non-null  float64
 1   Application Date      1048575 non-null  object 
 2   Loan Title            1048546 non-null  object 
 3   Risk_Score            781631 non-null   float64
 4   Debt-To-Income Ratio  1048575 non-null  object 
 5   Zip Code              1048531 non-null  object 
 6   State                 1048554 non-null  object 
 7   Employment Length     1026235 non-null  object 
 8   Policy Code           1048575 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


missingness

In [5]:
# accepted
for col in accepted.columns:
    print(f'{col}: {accepted[col].isnull().sum()}')

loan_amnt: 33
issue_d: 33
title: 23358
last_fico_range_high: 33
last_fico_range_low: 33
annual_inc: 37
installment: 33
zip_code: 34
addr_state: 33
emp_length: 146940
policy_code: 33


In [6]:
# rejected
for col in rejected.columns:
    print(f'{col}: {rejected[col].isnull().sum()}')

Amount Requested: 0
Application Date: 0
Loan Title: 29
Risk_Score: 266944
Debt-To-Income Ratio: 0
Zip Code: 44
State: 21
Employment Length: 22340
Policy Code: 0


accepted df cleaning

In [7]:
accepted_df = accepted.copy()
accepted_df['title'] = accepted_df['title'].str.replace('_', ' ').str.lower()

accepted_df['risk_score'] = (accepted_df['last_fico_range_high'] + accepted_df['last_fico_range_low']) / 2

accepted_df['annual_inc'] = np.where(accepted_df['annual_inc'] == 0, np.nan, accepted_df['annual_inc'])
accepted_df.dropna(subset = ['annual_inc'], inplace = True)
accepted_df['debt_to_income_ratio'] = accepted_df['installment'] / (accepted_df['annual_inc'] / 12)

accepted_df['year'] = accepted_df['issue_d'].str[-4:].astype('int')

accepted_df['status'] = 1

accepted_df.drop(['issue_d','last_fico_range_high','last_fico_range_low','installment','annual_inc'],
                 axis = 1, inplace = True)

accepted_df = accepted_df.rename(columns = {'loan_amnt':'amount', 'title':'title',
                                            'risk_score':'risk_score','zip_code':'zip', 'addr_state':'state',
                                            'emp_length':'length_of_emp','debt_to_income_ratio':'debt_to_income_ratio',
                                            'policy_code':'policy_code','year':'year','status':'status'})

In [8]:
accepted_df.sample(10)

,amount,title,zip,state,length_of_emp,policy_code,risk_score,debt_to_income_ratio,year,status
107141,20000.0,credit card refinancing,908xx,CA,5 years,1.0,752.0,0.026965,2015,1
1919541,2400.0,motorcycle,115xx,NY,10+ years,1.0,652.0,0.028163,2012,1
1075504,11200.0,debt consolidation,422xx,KY,9 years,1.0,647.0,0.107228,2016,1
235626,8500.0,debt consolidation,296xx,SC,4 years,1.0,707.0,0.049305,2015,1
209201,8400.0,debt consolidation,318xx,GA,10+ years,1.0,577.0,0.026779,2015,1
1447011,10000.0,credit card refinancing,183xx,PA,10+ years,1.0,642.0,0.035828,2018,1
32514,7000.0,debt consolidation,212xx,MD,1 year,1.0,827.0,0.050594,2015,1
2016053,7000.0,debt consolidation,273xx,NC,10+ years,1.0,682.0,0.033176,2016,1
699269,20800.0,credit card refinancing,662xx,KS,5 years,1.0,747.0,0.091194,2016,1
1336060,1500.0,home improvement,190xx,PA,6 years,1.0,612.0,0.004982,2014,1


In [26]:
accepted_df.groupby('year')['title'].count()

year
2007       599
2008      2392
2009      5280
2010     12526
2011     21721
2012     53365
2013    134809
2014    235629
2015    420961
2016    411182
2017    443086
2018    494127
Name: title, dtype: int64

In [9]:
accepted_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2258997 entries, 0 to 2260698
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   amount                float64
 1   title                 object 
 2   zip                   object 
 3   state                 object 
 4   length_of_emp         object 
 5   policy_code           float64
 6   risk_score            float64
 7   debt_to_income_ratio  float64
 8   year                  int32  
 9   status                int64  
dtypes: float64(4), int32(1), int64(1), object(4)
memory usage: 181.0+ MB


rejected df cleaning

In [10]:
rejected_df = rejected.copy()

rejected_df['Loan Title'] = rejected_df['Loan Title'].str.replace('_', ' ').str.lower()

rejected_df['year'] = rejected_df['Application Date'].str[-4:].astype('int')

rejected_df['debt_to_income_ratio'] = rejected_df['Debt-To-Income Ratio'].str.replace('%','').astype('float')/100

rejected_df['status'] = 0

rejected_df = rejected_df.drop(['Application Date','Debt-To-Income Ratio'], axis = 1)

rejected_df = rejected_df.rename(columns = {'Amount Requested':'amount', 'Loan Title':'title',
                                            'Risk_Score':'risk_score','Zip Code':'zip', 'State':'state',
                                            'Employment Length':'length_of_emp',
                                            'Policy Code':'policy_code','year':'year','status':'status'})

In [11]:
rejected_df.sample(10)

,amount,title,risk_score,zip,state,length_of_emp,policy_code,year,debt_to_income_ratio,status
989630,3000.0,other,NaN,921xx,CA,< 1 year,0,2016,0.1458,0
530037,1500.0,medical,659.0,920xx,CA,< 1 year,0,2012,0.1913,0
625947,5000.0,debt consolidation,588.0,949xx,CA,< 1 year,0,2012,0.3709,0
148500,4500.0,educational,702.0,421xx,KY,< 1 year,0,2010,0.3880,0
115900,1000.0,moving,0.0,970xx,OR,< 1 year,0,2010,0.0000,0
125006,13000.0,consolidate,737.0,941xx,CA,6 years,0,2010,0.0831,0
808635,10000.0,other,NaN,802xx,CO,< 1 year,0,2016,0.0068,0
151693,2000.0,moving,541.0,080xx,NJ,< 1 year,0,2010,0.0000,0
955636,6500.0,debt consolidation,NaN,157xx,PA,< 1 year,0,2016,0.2939,0
220117,10000.0,car,634.0,260xx,WV,10+ years,0,2011,0.0603,0


In [28]:
rejected_df.groupby('year')['title'].count()

year
2007      5274
2008     25595
2009     56981
2010    112560
2011    217792
2012    337275
2016    293069
Name: title, dtype: int64

In [12]:
rejected_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   amount                1048575 non-null  float64
 1   title                 1048546 non-null  object 
 2   risk_score            781631 non-null   float64
 3   zip                   1048531 non-null  object 
 4   state                 1048554 non-null  object 
 5   length_of_emp         1026235 non-null  object 
 6   policy_code           1048575 non-null  int64  
 7   year                  1048575 non-null  int32  
 8   debt_to_income_ratio  1048575 non-null  float64
 9   status                1048575 non-null  int64  
dtypes: float64(3), int32(1), int64(2), object(4)
memory usage: 76.0+ MB


accepted_df + rejected_df

In [13]:
lc_df = pd.concat([accepted_df, rejected_df], axis = 0)

In [14]:
lc_df.sample(10)

,amount,title,zip,state,length_of_emp,policy_code,risk_score,debt_to_income_ratio,year,status
601780,14000.0,debt consolidation,601xx,IL,NaN,1.0,652.0,0.072682,2017,1
1774938,21000.0,my new life,785xx,TX,3 years,1.0,697.0,0.157054,2013,1
922709,5000.0,vacation,983xx,WA,2 years,1.0,532.0,0.033617,2017,1
1582743,5000.0,debt consolidation,339xx,FL,< 1 year,1.0,667.0,0.051447,2018,1
631624,40000.0,debt consolidation,481xx,MI,10+ years,1.0,602.0,0.159838,2017,1
206494,20000.0,debt consolidation,604xx,IL,< 1 year,0.0,670.0,0.170000,2011,0
321993,12000.0,debt consolidation,080xx,NJ,NaN,1.0,567.0,0.048855,2015,1
1732592,17600.0,credit card refinancing,350xx,AL,5 years,1.0,752.0,0.100686,2017,1
1705889,7000.0,debt consolidation,926xx,CA,NaN,1.0,747.0,0.079077,2017,1
686864,10000.0,small business,206xx,MD,< 1 year,0.0,637.0,0.254100,2012,0


adding region columns

In [15]:
region = pd.read_csv('states.csv')

In [16]:
region.sample(5)

,State,State Code,Region,Division
46,Vermont,VT,Northeast,New England
5,Colorado,CO,West,Mountain
26,Montana,MT,West,Mountain
29,Nebraska,NE,Midwest,West North Central
19,Massachusetts,MA,Northeast,New England


In [17]:
lc_df = lc_df.merge(region, how = 'left', left_on = 'state', right_on = 'State Code')

In [18]:
lc_df.drop(['zip','State','State Code'], axis = 1, inplace = True)

In [19]:
lc_df.columns = lc_df.columns.str.lower()

In [20]:
for col in lc_df.columns:
    print(f'{col}: {lc_df[col].isnull().sum()}')

amount: 0
title: 23349
state: 21
length_of_emp: 167680
policy_code: 0
risk_score: 266944
debt_to_income_ratio: 0
year: 0
status: 0
region: 21
division: 21


In [21]:
lc_df.dropna(subset = ['state'], inplace = True)

In [22]:
lc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3307551 entries, 0 to 3307571
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   amount                float64
 1   title                 object 
 2   state                 object 
 3   length_of_emp         object 
 4   policy_code           float64
 5   risk_score            float64
 6   debt_to_income_ratio  float64
 7   year                  int32  
 8   status                int64  
 9   region                object 
 10  division              object 
dtypes: float64(4), int32(1), int64(1), object(5)
memory usage: 290.2+ MB


export

In [23]:
lc_df.to_csv('lc_cleaned.csv', index = False)